In [2]:
import sys
sys.path.append("/home/jovyan/TF_NEW/tf-transformers/src/")



In [3]:
import os
import tempfile
import json
import glob
import datasets
import shutil
import tensorflow as tf

from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf

from tf_transformers.data import TFReader, TFWriter
from tf_transformers.models import Classification_Model
from tf_transformers.losses import cross_entropy_loss_for_classification
from model import get_model, get_tokenizer, get_optimizer, get_trainer

In [4]:
with initialize(config_path="conf/"):
    cfg = compose(config_name="config", overrides=["data.take_sample=true", "+glue=mrpc"])
    print(cfg)

{'data': {'train_batch_size': 32, 'eval_batch_size': 64, 'take_sample': True, 'max_seq_length': 128}, 'trainer': {'type': 'gpu', 'dtype': 'fp32', 'num_gpus': 2, 'tpu_address': None, 'epochs': 3, 'strategy': 'mirrored'}, 'optimizer': {'learning_rate': 2e-05, 'loss_type': None}, 'model': {'is_training': True, 'use_dropout': True}, 'glue': {'task': {'name': 'mrpc'}, 'data': {'name': 'mrpc', 'num_classes': 2}}}


In [5]:
# Steps

# 1. Download the data
# 2. Prepare TFRecords
# 3. Read TFrecords to tf.data
# 4. Train the model

In [6]:
def get_classification_model(num_classes, return_all_layer_outputs, is_training, use_dropout):
        
    def model_fn():
        model = get_model(return_all_layer_outputs, is_training, use_dropout)
        classification_model = Classification_Model(model,
                                                    num_classes, 
                                                    use_all_layers=return_all_layer_outputs, 
                                                    is_training=is_training, 
                                                    use_dropout=use_dropout)
        classification_model = classification_model.get_model()
        return classification_model
    return model_fn

In [7]:
# Convert data to features using specific length
# into a temp dir (and log it as well for monitoring)

def get_dataset(data, batch_size, tokenizer, max_seq_length, mode, tfrecord_dir, take_sample=False):
    
    if mode not in ["train", "eval"]:
        raise ValueError("Inavlid mode `{}` specified. Available mode is ['train', 'eval']".format(mode))
    
    def get_tfrecord_example(data):
        result = {}
        for f in data:
            input_ids_s1 = [tokenizer.cls_token] + tokenizer.tokenize(f['sentence1'])[: max_seq_length-2] + [tokenizer.sep_token] # -2 to add CLS and SEP
            input_ids_s1 = tokenizer.convert_tokens_to_ids(input_ids_s1)
            input_type_ids_s1 = [0] * len(input_ids_s1) # 0 for s1

            input_ids_s2 = tokenizer.tokenize(f['sentence2'])[: max_seq_length-1] + [tokenizer.sep_token] # -1 to add SEP
            input_ids_s2 = tokenizer.convert_tokens_to_ids(input_ids_s2)
            input_type_ids_s2 = [1] * len(input_ids_s2)
            
            # concatanate two sentences
            input_ids =  input_ids_s1 + input_ids_s2
            input_type_ids = input_type_ids_s1 + input_type_ids_s2
            input_mask = [1] * len(input_ids) # 1 for s2
            
            result = {}
            result['input_ids'] = input_ids
            result['input_mask'] = input_mask
            result['input_type_ids'] = input_type_ids

            result['labels'] = f['label']
            yield result
            
    schema = {
        "input_ids": ("var_len", "int"),
        "input_mask": ("var_len", "int"),
        "input_type_ids": ("var_len", "int"),
        "labels": ("var_len", "int"),
    }
    
    # Create a temp dir
    if mode == "train":
        # Write tf records
        train_data_dir = os.path.join(tfrecord_dir,"train")        
        tfrecord_filename = 'mrpc'
        tfwriter = TFWriter(schema=schema, 
                            file_name=tfrecord_filename, 
                            model_dir=train_data_dir,
                            tag='train',
                            overwrite=False
                     )
        data_train = data['train']
        # Take sample
        if take_sample:
            data_train = data_train.select(range(500))
            
        tfwriter.process(parse_fn=get_tfrecord_example(data_train))
        
        # Read tfrecord to dataset
        schema = json.load(open("{}/schema.json".format(train_data_dir)))
        stats  = json.load(open('{}/stats.json'.format(train_data_dir)))
        all_files = glob.glob("{}/*.tfrecord".format(train_data_dir))
        tf_reader = TFReader(schema=schema, 
                            tfrecord_files=all_files)

        x_keys = ['input_ids', 'input_type_ids', 'input_mask']
        y_keys = ['labels']
        train_dataset = tf_reader.read_record(auto_batch=True, 
                                           keys=x_keys,
                                           batch_size=batch_size, 
                                           x_keys = x_keys, 
                                           y_keys = y_keys,
                                           shuffle=True, 
                                           drop_remainder=True
                                          )
        return train_dataset, stats['total_records']
    if mode == "eval":
        # Write tfrecords
        eval_data_dir = os.path.join(tfrecord_dir,"eval")
        tfrecord_filename = 'mrpc'
        tfwriter = TFWriter(schema=schema, 
                            file_name=tfrecord_filename, 
                            model_dir=eval_data_dir,
                            tag='dev',
                            overwrite=False
                            )
        data_eval = data['validation']
        # Take sample
        if take_sample:
            data_eval = data_eval.select(range(500))
        tfwriter.process(parse_fn=get_tfrecord_example(data_eval))
        
        
        # Read tfrecord to dataset
        schema = json.load(open("{}/schema.json".format(eval_data_dir)))
        stats  = json.load(open('{}/stats.json'.format(eval_data_dir)))
        all_files = glob.glob("{}/*.tfrecord".format(eval_data_dir))
        tf_reader = TFReader(schema=schema, 
                            tfrecord_files=all_files)

        x_keys = ['input_ids', 'input_type_ids', 'input_mask']
        y_keys = ['labels']
        eval_dataset = tf_reader.read_record(auto_batch=True, 
                                           keys=x_keys,
                                           batch_size=batch_size, 
                                           x_keys = x_keys, 
                                           y_keys = y_keys,
                                           shuffle=False, 
                                           drop_remainder=False
                                          )
        return eval_dataset, stats['total_records']

In [8]:
def get_loss(loss_type):

    if loss_type and loss_type == 'joint':

        def loss_fn(y_true_dict, y_pred_dict):
            """Joint loss over all layers"""
            loss_dict = {}
            loss_holder = []
            for layer_count, per_layer_output in enumerate(y_pred_dict['class_logits']):

                loss = cross_entropy_loss_for_classification(
                    labels=tf.squeeze(y_true_dict['labels'], axis=1),
                    logits=per_layer_output
                )
                loss_dict['loss_{}'.format(layer_count + 1)] = loss
                loss_holder.append(loss)
            # Mean over batch
            loss_dict['loss'] = tf.reduce_mean(loss_holder, axis=0)
            return loss_dict

    else:

        def loss_fn(y_true_dict, y_pred_dict):
            """last layer loss"""
            loss_dict = {}
            loss = cross_entropy_loss_for_classification(
                labels=tf.squeeze(y_true_dict['labels'], axis=1),
                logits=y_pred_dict['class_logits']
            )
            loss_dict['loss'] = loss
            return loss_dict

    return loss_fn

In [9]:
cfg

{'data': {'train_batch_size': 32, 'eval_batch_size': 64, 'take_sample': True, 'max_seq_length': 128}, 'trainer': {'type': 'gpu', 'dtype': 'fp32', 'num_gpus': 2, 'tpu_address': None, 'epochs': 3, 'strategy': 'mirrored'}, 'optimizer': {'learning_rate': 2e-05, 'loss_type': None}, 'model': {'is_training': True, 'use_dropout': True}, 'glue': {'task': {'name': 'mrpc'}, 'data': {'name': 'mrpc', 'num_classes': 2}}}

In [10]:

# Data specific configuration
max_seq_len = cfg.data.max_seq_length
take_sample = cfg.data.take_sample
max_seq_length = cfg.data.max_seq_length
train_batch_size = cfg.data.train_batch_size
eval_batch_size  = cfg.data.eval_batch_size

# Trainer specifics
device = cfg.trainer.type
num_gpus = cfg.trainer.num_gpus
tpu_address = cfg.trainer.tpu_address
dtype = cfg.trainer.dtype
epochs = cfg.trainer.epochs
strategy = cfg.trainer.strategy

# Optimizer
learning_rate = cfg.optimizer.learning_rate
loss_type = cfg.optimizer.loss_type
return_all_layer_outputs = False
if loss_type and loss_type == 'joint':
    return_all_layer_outputs = True

# Core data specifics
data_name = cfg_task.glue.data.name
num_classes = cfg_task.glue.data.num_classes

# Model specific
is_training = cfg.model.is_training
use_dropout = cfg.model.use_dropout

In [11]:
# Load tokenizer
tokenizer = get_tokenizer()

# Load data
data = datasets.load_dataset("glue", data_name)
tfrecord_dir = tempfile.mkdtemp()

train_dataset, total_train_examples = get_dataset(data, train_batch_size,tokenizer, max_seq_length, "train", tfrecord_dir, take_sample)
eval_dataset, total_eval_examples  = get_dataset(data, eval_batch_size,tokenizer, max_seq_len, "eval", tfrecord_dir, take_sample)

Reusing dataset glue (/home/jovyan/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
INFO:absl:Total individual observations/examples written is 500 in 0.33753085136413574 seconds
INFO:absl:All writer objects closed
INFO:absl:Total individual observations/examples written is 500 in 0.3376038074493408 seconds
INFO:absl:All writer objects closed


In [12]:
# Load optimizer
optimizer_fn = get_optimizer(learning_rate, total_train_examples, train_batch_size, epochs)

# Load trainer
# trainer = get_trainer(device, dtype, strategy, num_gpus, tpu_address)


In [13]:
# Load model function
model_fn = get_classification_model(num_classes, 
                                 return_all_layer_outputs, is_training, use_dropout)
# Load loss function 
train_loss_fn = get_loss(loss_type)

In [14]:
for (batch_inputs, batch_labels) in train_dataset.take(1):
    print(batch_inputs['input_ids'].shape, batch_labels['labels'].shape)

(32, 95) (32, 1)


In [14]:
class Callback():
    
    def __init__(self):
        pass
    
    def call(trainer_kwargs):
        
        for k, v in trainer_kwargs.items():
            print(k, '-->', v)
callback = Callback()

In [15]:
# coding=utf-8
# Copyright 2021 TF-Transformers Authors.
# All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
import tensorflow as tf
import tqdm
from absl import logging

from tf_transformers.core import keras_utils
from tf_transformers.core.distribute_utils import get_distribution_strategy
from tf_transformers.core.performance_utils import (
    configure_optimizer,
    get_tf_dtype,
    is_float16,
    set_mixed_precision_policy,
)


def flat_metric_dict(metric_dict):
    """Flatten the dict"""
    dict_flatten = {}
    dict_flatten['steps'] = list(metric_dict.keys())
    for _key, value in metric_dict.items():
        for sub_key, sub_value in value.items():
            if sub_key not in dict_flatten:
                dict_flatten[sub_key] = [sub_value]
            else:
                dict_flatten[sub_key].append(sub_value)
    return dict_flatten


def save_model_checkpoints(model, overwrite_checkpoint_dir, model_checkpoint_dir, max_number_of_models):
    # Model checkpoint
    if not overwrite_checkpoint_dir:
        import os

        if os.path.exists(model_checkpoint_dir):
            raise FileExistsError("Model directory exists")

    checkpoint = tf.train.Checkpoint(model=model)
    manager = tf.train.CheckpointManager(checkpoint, directory=model_checkpoint_dir, max_to_keep=max_number_of_models)
    return manager


def get_loss_metric_dict(training_loss_names, validation_loss_names):

    training_loss_dict_metric = {name: tf.keras.metrics.Mean(name, dtype=tf.float32) for name in training_loss_names}
    training_loss_dict_metric["learning_rate"] = tf.keras.metrics.Mean(
        "learning_rate", dtype=tf.float32
    )  # We store learning rate here and reset after every global steps

    validation_loss_dict_metric = {}
    if validation_loss_names:
        validation_loss_dict_metric = {
            name: tf.keras.metrics.Mean(name, dtype=tf.float32) for name in validation_loss_names
        }

    return training_loss_dict_metric, validation_loss_dict_metric


def get_and_reset_metric_from_dict(metric_dict):
    if not metric_dict:
        return {}
    metric_result = {name: metric.result().numpy() for name, metric in metric_dict.items()}
    for _name, metric in metric_dict.items():
        metric.reset_states()
    return metric_result


def get_tensorboard_writers(model_checkpoint_dir):
    train_log_dir = model_checkpoint_dir + "/logs/train"
    test_log_dir = model_checkpoint_dir + "/logs/dev"
    train_summary_writer = tf.summary.create_file_writer(train_log_dir)
    test_summary_writer = tf.summary.create_file_writer(test_log_dir)
    return train_summary_writer, test_summary_writer


def train_and_eval(
    model,
    optimizer,
    strategy,
    epochs,
    steps_per_epoch,
    steps_per_call,
    train_dataset_iter,
    train_loss_fn,
    GLOBAL_BATCH_SIZE,
    training_loss_dict_metric,
    validation_dataset_distributed,
    validation_loss_fn,
    validation_loss_dict_metric,
    validation_interval_steps,
    mixed_precision,
    callbacks,
    callbacks_interval_steps,
    trainer_kwargs,
    checkpoint_manager,
    model_checkpoint_dir,
    model_save_interval_steps,
):
    def save_model(epoch_end=False):
        if not epoch_end:
            if model_save_interval_steps:
                if global_step % model_save_interval_steps == 0:
                    checkpoint_manager.save()
                    logging.info("Model saved at step {}".format(global_step))
        else:
            checkpoint_manager.save()
            logging.info("Model saved at epoch {}".format(epoch))

    # @tf.function(experimental_relax_shapes=True)
    def write_metrics(metric_dict, writer, step):
        # @tf.function
        def _write(step):
            # other model code would go here
            with writer.as_default():
                for name, result in metric_dict.items():
                    tf.summary.scalar(name, result, step=step)

        _write(step)
        writer.flush()

    def compute_loss(batch_labels, model_outputs):
        """Loss computation which takes care of loss reduction based on GLOBAL_BATCH_SIZE"""
        per_example_loss = train_loss_fn(batch_labels, model_outputs)
        per_example_loss_averaged = {}
        # Inplace update
        # Avergae loss per global batch size , recommended
        for name, loss in per_example_loss.items():
            per_example_loss_averaged[name] = tf.nn.compute_average_loss(loss, global_batch_size=GLOBAL_BATCH_SIZE)
        return per_example_loss_averaged

    def compute_loss_valid(batch_labels, model_outputs):
        """Validation Loss computation which takes care of loss reduction based on GLOBAL_BATCH_SIZE"""
        per_example_loss = validation_loss_fn(batch_labels, model_outputs)
        per_example_loss_averaged = {}
        # Inplace update
        # Avergae loss per global batch size , recommended
        for name, loss in per_example_loss.items():
            per_example_loss_averaged[name] = tf.nn.compute_average_loss(loss, global_batch_size=GLOBAL_BATCH_SIZE)
        return per_example_loss_averaged

    # Train Functions
    @tf.function
    def do_train(iterator):
        """The step function for one training step"""

        def train_step(dist_inputs):
            """The computation to run on each device."""
            batch_inputs, batch_labels = dist_inputs
            with tf.GradientTape() as tape:
                model_outputs = model(batch_inputs)
                loss = compute_loss(batch_labels, model_outputs)
                tf.debugging.check_numerics(loss['loss'], message='Loss value is either NaN or inf')
                if isinstance(optimizer, tf.keras.mixed_precision.LossScaleOptimizer):
                    loss_scaled = {name: optimizer.get_scaled_loss(loss_value) for name, loss_value in loss.items()}
                # TODO
                # Scales down the loss for gradients to be invariant from replicas.
                # loss = loss / strategy.num_replicas_in_sync
            if mixed_precision:
                scaled_gradients = tape.gradient(loss_scaled["loss"], model.trainable_variables)
                grads = optimizer.get_unscaled_gradients(scaled_gradients)
            else:
                grads = tape.gradient(loss["loss"], model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))
            # training_loss.update_state(loss * strategy.num_replicas_in_sync)
            return loss

        for _ in tf.range(tf.convert_to_tensor(steps_per_call)):
            dist_inputs = next(iterator)
            loss = strategy.run(train_step, args=(dist_inputs,))
            # strategy reduce
            loss = {
                name: strategy.reduce(tf.distribute.ReduceOp.MEAN, loss_value, axis=None)
                for name, loss_value in loss.items()
            }
            for name, loss_value in loss.items():
                training_loss = training_loss_dict_metric[name]
                training_loss.update_state(loss_value)
            # Get current learning rate
            if isinstance(optimizer, tf.keras.mixed_precision.LossScaleOptimizer):
                current_lr = optimizer._optimizer._decayed_lr(tf.float32)
            else:
                current_lr = optimizer._decayed_lr(tf.float32)
            training_loss_dict_metric["learning_rate"].update_state(current_lr)
            # training_result = get_and_reset_metric_from_dict(training_loss_dict_metric)

    # do validation
    def do_validation(validation_dataset_distributed):
        """Validation step"""

        @tf.function
        def _validate_step(dist_inputs):

            batch_inputs, batch_labels = dist_inputs
            model_outputs = model(batch_inputs)
            loss = compute_loss_valid(batch_labels, model_outputs)
            return loss

        if not epoch_end:
            if (
                validation_dataset_distributed
                and validation_loss_fn
                and validation_interval_steps
                and (global_step % validation_interval_steps == 0)
            ):
                logging.info("Validation in progress at step {} . . . .".format(global_step))
                with tqdm.tqdm(validation_dataset_distributed, unit=" Val batch ") as val_batches:
                    for dist_inputs in val_batches:
                        loss = strategy.run(_validate_step, args=(dist_inputs,))
                        for name, loss_value in loss.items():
                            loss_value = strategy.reduce(tf.distribute.ReduceOp.SUM, loss_value, axis=None)
                            validation_loss = validation_loss_dict_metric[name]
                            validation_loss.update_state(loss_value)

                validation_result = get_and_reset_metric_from_dict(validation_loss_dict_metric)
                validation_history[global_step] = validation_result
                write_metrics(validation_result, val_summary_writer, global_step)
                logging.info("Validation result at step {}".format(validation_result))
                print("\n")
        else:
            if validation_dataset_distributed and validation_loss_fn:
                logging.info("Validation in progress at epoch end {} . . . .".format(epoch))
                with tqdm.tqdm(validation_dataset_distributed, unit=" Val batch ") as val_batches:
                    for dist_inputs in val_batches:
                        loss = strategy.run(_validate_step, args=(dist_inputs,))
                        for name, loss_value in loss.items():
                            loss_value = strategy.reduce(tf.distribute.ReduceOp.SUM, loss_value, axis=None)
                            validation_loss = validation_loss_dict_metric[name]
                            validation_loss.update_state(loss_value)

                validation_result = get_and_reset_metric_from_dict(validation_loss_dict_metric)
                write_metrics(validation_result, val_summary_writer, global_step)
                # validation_history[global_step] = validation_result
                logging.info("Validation result at epoch {} is {}".format(epoch, validation_result))
                print("\n")

    def do_callbacks(callbacks):
        """Call callbacks"""
        if not epoch_end:
            callback_scores = None
            if callbacks and callbacks_interval_steps:
                logging.info("Callbacks in progress at step {} . . . .".format(global_step))
                callback_scores = []
                for callback, callback_steps in zip(callbacks, callbacks_interval_steps):
                    if callback_steps and (global_step % callback_steps == 0):
                        score = callback(trainer_kwargs)
                        callback_scores.append(score)
                    else:
                        callback_scores.append(None)
            return callback_scores
        else:
            callback_scores = None
            if callbacks:
                logging.info("Callbacks in progress at epoch end {} . . . .".format(epoch))
                callback_scores = []
                for callback in callbacks:
                    score = callback(trainer_kwargs)
                    callback_scores.append(score)

                    # Try to write a callback scores (only on epoch end)
                    # If we are returning a dict like {'exact_match': 81} or
                    # {'rougue-1': 30} etc . . . .
                    if score and isinstance(score, dict):
                        write_metrics(score, val_summary_writer, epoch)
            return callback_scores

    # Loop starts here
    # Get Tensorboard writers
    train_summary_writer, val_summary_writer = get_tensorboard_writers(model_checkpoint_dir)
    validation_history = {}
    training_history = {}
    global_step = 0
    epoch_end = False
    STEPS = steps_per_epoch // steps_per_call
    print("STEPS", STEPS)
    for epoch in range(1, epochs + 1):
        # start_epoch_time = time.time()
        with tqdm.trange(STEPS, unit="batch ") as tepoch:
            for step in tepoch:
                steps_covered = (step + 1) * steps_per_call
                global_step += steps_per_call
                print("Started epoch {} and step {}".format(epoch, global_step))
                tepoch.set_description(
                    "Epoch {}/{} --- Step {}/{} --- ".format(epoch, epochs, steps_covered, steps_per_epoch)
                )
                # Call Train
                do_train(train_dataset_iter)
                print("Train done")
                # Call Validation
                do_validation(validation_dataset_distributed)
                print("Val done")
                # Call Callbacks
                callback_scores = do_callbacks(callbacks)

                # Train Metrics
                training_result = get_and_reset_metric_from_dict(training_loss_dict_metric)
                training_history[global_step] = training_result
                write_metrics(training_result, train_summary_writer, global_step)
                # training_result["learning_rate"] = learning_rate_holder.result().numpy()
                # learning_rate_holder.reset_states()
                tepoch.set_postfix(**training_result)

                # Save model
                save_model()

        # Do after every epoch
        epoch_end = True
        save_model(epoch_end)
        #do_validation(validation_dataset_distributed)
        #callback_scores = do_callbacks(callbacks)
        epoch_end = False

    # Flatten the results
    training_history = flat_metric_dict(training_history)
    validation_history = flat_metric_dict(validation_history)
    return training_history, validation_history, callback_scores


class GPUTrainer:
    def __init__(
        self,
        distribution_strategy,
        num_gpus=0,
        all_reduce_alg=None,
        num_packs=1,
        tpu_address=None,
        dtype='fp32',
        loss_scale='dynamic',
    ):

        self.distribution_strategy = get_distribution_strategy(
            distribution_strategy=distribution_strategy,
            num_gpus=num_gpus,
            all_reduce_alg=all_reduce_alg,
            num_packs=num_packs,
            tpu_address=tpu_address,
        )

        self.num_replicas = self.distribution_strategy.num_replicas_in_sync
        self._dtype = get_tf_dtype(dtype)

        # Setting dtype policy
        set_mixed_precision_policy(self._dtype)
        self.use_float16 = is_float16(self._dtype)
        self.loss_scale = loss_scale

        # # TODO
        # if self.use_tpu:
        # params["num_replicas"] = self.distribution_strategy.num_replicas_in_sync
        # else:
        # logging.info("Running transformer with num_gpus = %d", num_gpus)

        # Add keras utils threads

    def run(
        self,
        model_fn,
        optimizer_fn,
        train_dataset,
        train_loss_fn,
        epochs,
        steps_per_epoch,
        model_checkpoint_dir,
        batch_size,
        training_loss_names=None,
        validation_loss_names=None,
        validation_dataset=None,
        validation_loss_fn=None,
        validation_interval_steps=None,
        steps_per_call=100,
        enable_xla=True,
        callbacks=None,
        callbacks_interval_steps=None,
        overwrite_checkpoint_dir=False,
        max_number_of_models=10,
        model_save_interval_steps=None,
        repeat_dataset=True,
        latest_checkpoint=None,
    ):

        if steps_per_epoch:
            logging.info("Make sure `steps_per_epoch` should be less than or equal to number of batches in dataset.")
        if callbacks:
            if callbacks_interval_steps is None:
                callbacks_interval_steps = [None for callback in callbacks]
            assert len(callbacks) == len(callbacks_interval_steps)

        # Enable XLA
        keras_utils.set_session_config(enable_xla=enable_xla)
        logging.info("Policy: ----> {}".format(keras_utils.get_policy_name()))
        logging.info("Strategy: ---> {}".format(self.distribution_strategy))
        logging.info("Num GPU Devices: ---> {}".format(self.distribution_strategy.num_replicas_in_sync))

        tf.keras.backend.clear_session()

        # Under Strategy Scope
        with self.distribution_strategy.scope():
            # Model
            model = model_fn()

            # Optimizer
            optimizer = optimizer_fn()

            optimizer = configure_optimizer(optimizer, use_float16=self.use_float16, loss_scale=self.loss_scale)

        # We use this to avoid inferring names from loss functions
        _training_loss_names = ['loss']
        _validation_loss_names = ['loss']
        if training_loss_names:
            _training_loss_names += training_loss_names
        if validation_loss_names:
            _validation_loss_names += validation_loss_names
        # Make unique names
        training_loss_names = list(set(_training_loss_names))
        validation_loss_names = list(set(_validation_loss_names))
        # Checkpoint manager
        checkpoint_manager = save_model_checkpoints(
            model, overwrite_checkpoint_dir, model_checkpoint_dir, max_number_of_models
        )

        # Try to load latest checkpoint
        model.load_checkpoint(checkpoint_dir=model_checkpoint_dir, checkpoint_path=latest_checkpoint, opt=optimizer)

        # Get metric dicts before distributing the dataset
        # ddistributed datasets has no attribute .take
        training_loss_dict_metric, validation_loss_dict_metric = get_loss_metric_dict(
            training_loss_names, validation_loss_names
        )
        # Distribute dataset
        if not repeat_dataset:
            train_dataset_distributed = self.distribution_strategy.experimental_distribute_dataset(
                train_dataset.repeat(epochs + 1)
            )
        else:
            train_dataset_distributed = self.distribution_strategy.experimental_distribute_dataset(
                train_dataset.repeat()
            )
        validation_dataset_distributed = None
        if validation_dataset:
            validation_dataset_distributed = self.distribution_strategy.experimental_distribute_dataset(
                validation_dataset
            )

        # Make train dataset iterator
        train_dataset_distributed = iter(train_dataset_distributed)

        history = {}
        training_history, validation_history, callback_scores = train_and_eval(
            model,
            optimizer,
            self.distribution_strategy,
            epochs,
            steps_per_epoch,
            steps_per_call,
            train_dataset_distributed,
            train_loss_fn,
            batch_size,
            training_loss_dict_metric,
            validation_dataset_distributed,
            validation_loss_fn,
            validation_loss_dict_metric,
            validation_interval_steps,
            self.use_float16,
            callbacks,
            callbacks_interval_steps,
            locals(),
            checkpoint_manager,
            model_checkpoint_dir,
            model_save_interval_steps,
        )
        history['training_history'] = training_history
        history['validation_hsitory'] = validation_history
        history['callbacks'] = callback_scores

        # Save json
        return history


In [16]:
distribution_strategy = "mirrored"
num_gpus = 2

In [17]:
trainer = GPUTrainer(
        distribution_strategy,
        num_gpus=num_gpus,
        all_reduce_alg=None,
        num_packs=1,
        tpu_address=None,
        dtype='fp32',
        loss_scale='dynamic',
    )

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


In [19]:
model_checkpoint_dir = "/tmp/model_ckpt/"
history = trainer.run(
    model_fn = model_fn,
    optimizer_fn = optimizer_fn,
    train_dataset = train_dataset,
    train_loss_fn = train_loss_fn,
    epochs = 2,
    steps_per_epoch = 100,
    model_checkpoint_dir=model_checkpoint_dir,
    batch_size=train_batch_size,
    training_loss_names=None,
    validation_loss_names=None,
    validation_dataset=eval_dataset,
    validation_loss_fn=train_loss_fn,
    validation_interval_steps=None,
    steps_per_call=1,
    enable_xla=False,
    callbacks=[callback],
    callbacks_interval_steps=None,
    overwrite_checkpoint_dir=True,
    max_number_of_models=10,
    model_save_interval_steps=None,
    repeat_dataset=True,
    latest_checkpoint=None,
)

INFO:absl:Make sure `steps_per_epoch` should be less than or equal to number of batches in dataset.
INFO:absl:Policy: ----> float32
INFO:absl:Strategy: ---> <tensorflow.python.distribute.mirrored_strategy.MirroredStrategy object at 0x7ff612223790>
INFO:absl:Num GPU Devices: ---> 2
You are using a model of type albert to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
INFO:absl:Successful: Model checkpoints matched and loaded from /tmp/tf_transformers_cache/albert-base-v2/ckpt-1
INFO:absl:Using Adamw optimizer
INFO:absl:No checkpoint found in /tmp/model_ckpt/
Epoch 1/2 --- Step 1/100 --- :   0%|          | 0/100 [00:00<?, ?batch /s]

STEPS 100
Started epoch 1 and step 1


INFO:tensorflow:batch_all_reduce: 26 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:batch_all_reduce: 26 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1').


...

Train done
Val done
Started epoch 2 and step 197
Train done
Val done
Started epoch 2 and step 198


INFO:absl:Callbacks in progress at step 198 . . . .
Epoch 2/2 --- Step 100/100 --- :  99%|█████████▉| 99/100 [00:12<00:00,  8.01batch /s, learning_rate=0, loss=0.304]

Train done
Val done
Started epoch 2 and step 199
Train done
Val done
Started epoch 2 and step 200


INFO:absl:Callbacks in progress at step 200 . . . .
Epoch 2/2 --- Step 100/100 --- : 100%|██████████| 100/100 [00:12<00:00,  8.22batch /s, learning_rate=0, loss=0.278]
INFO:absl:Model saved at epoch 2


Train done
Val done


In [19]:
        # Under Strategy Scope
        with trainer.distribution_strategy.scope():
            # Model
            model = model_fn()

            # Optimizer
            optimizer = optimizer_fn()

            optimizer = configure_optimizer(optimizer, use_float16=False, loss_scale="dynamic")


You are using a model of type albert to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
INFO:absl:Successful: Model checkpoints matched and loaded from /tmp/tf_transformers_cache/albert-base-v2/ckpt-1
INFO:absl:Using Adamw optimizer


In [20]:
    def compute_loss(batch_labels, model_outputs):
        """Loss computation which takes care of loss reduction based on GLOBAL_BATCH_SIZE"""
        per_example_loss = train_loss_fn(batch_labels, model_outputs)
        per_example_loss_averaged = {}
        # Inplace update
        # Avergae loss per global batch size , recommended
        for name, loss in per_example_loss.items():
            per_example_loss_averaged[name] = tf.nn.compute_average_loss(loss, global_batch_size=GLOBAL_BATCH_SIZE)
        return per_example_loss_averaged

In [36]:
    # Train Functions
    @tf.function
    def do_train(iterator):
        """The step function for one training step"""

        def train_step(dist_inputs):
            """The computation to run on each device."""
            batch_inputs, batch_labels = dist_inputs
            with tf.GradientTape() as tape:
                model_outputs = model(batch_inputs)
                loss = compute_loss(batch_labels, model_outputs)
                tf.debugging.check_numerics(loss['loss'], message='Loss value is either NaN or inf')
                if isinstance(optimizer, tf.keras.mixed_precision.LossScaleOptimizer):
                    loss_scaled = {name: optimizer.get_scaled_loss(loss_value) for name, loss_value in loss.items()}
                # TODO
                # Scales down the loss for gradients to be invariant from replicas.
                # loss = loss / strategy.num_replicas_in_sync
            if mixed_precision:
                scaled_gradients = tape.gradient(loss_scaled["loss"], model.trainable_variables)
                grads = optimizer.get_unscaled_gradients(scaled_gradients)
            else:
                grads = tape.gradient(loss["loss"], model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))
            # training_loss.update_state(loss * strategy.num_replicas_in_sync)
            return loss

        for _ in tf.range(tf.convert_to_tensor(steps_per_call)):
            dist_inputs = next(iterator)
            loss = strategy.run(train_step, args=(dist_inputs,))
            # strategy reduce
            loss = {
                name: strategy.reduce(tf.distribute.ReduceOp.MEAN, loss_value, axis=None)
                for name, loss_value in loss.items()
            }
            
            t_loss.update_state(loss['loss'])
            # training_result = get_and_reset_metric_from_dict(training_loss_dict_metric)

In [22]:
train_dataset_distributed = trainer.distribution_strategy.experimental_distribute_dataset(train_dataset)
train_dataset_distributed = iter(train_dataset_distributed)

In [34]:
steps_per_call = 1
GLOBAL_BATCH_SIZE = 32
mixed_precision = False
strategy = trainer.distribution_strategy
t_loss = tf.keras.metrics.Mean("loss", dtype=tf.float32)

In [37]:
l = do_train(train_dataset_distributed)

INFO:tensorflow:batch_all_reduce: 26 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:batch_all_reduce: 26 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1').


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1').


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


In [38]:
t_loss.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.36904955>

INFO:absl:Make sure `steps_per_epoch` should be less than or equal to number of batches in dataset.
INFO:absl:XLA enabled
INFO:absl:Policy: ----> float32
INFO:absl:Strategy: ---> <tensorflow.python.distribute.mirrored_strategy.MirroredStrategy object at 0x7fbe2c4b1280>
INFO:absl:Num GPU Devices: ---> 2
You are using a model of type albert to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
INFO:absl:Successful: Model checkpoints matched and loaded from /tmp/tf_transformers_cache/albert-base-v2/ckpt-1
INFO:absl:Using Adamw optimizer
INFO:absl:No checkpoint found in /tmp/model_ckpt/
Epoch 1/2 --- Step 50/100 --- :   0%|          | 0/2 [00:00<?, ?batch /s]

STEPS 2
Started epoch 1 and step 50


INFO:tensorflow:batch_all_reduce: 26 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:batch_all_reduce: 26 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1').


...

INFO:tensorflow:batch_all_reduce: 26 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1').


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1').


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


In [ ]:
shutil.rmtree(tfrecord_dir)
shutil.rmtree(model_checkpoint_dir)